In [9]:
# Importa a biblioteca essencial para trabalhar com dados (planilhas)
import pandas as pd
import numpy as np

# -------------------------------------------------------------
# 1. Definição do Problema (Para o seu texto no Notebook)
# -------------------------------------------------------------

# **Justificativa de Negócio (Markdown):** O problema de Churn é a perda de clientes de uma empresa de telecomunicações. O principal objetivo é desenvolver um modelo preditivo que identifique, com antecedência, quais clientes têm maior probabilidade de cancelar seus serviços, permitindo à equipe de retenção agir proativamente. A prioridade é minimizar **Falsos Negativos** (perder um cliente sem aviso), que são mais custosos do que Falsos Positivos.

# -------------------------------------------------------------
# 2. Coleta e Entendimento dos Dados
# -------------------------------------------------------------

# Usamos a base de dados padrão IBM Telco Churn:
url_dados = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url_dados)

print("--- 10 Primeiras Linhas (Amostra de Dados) ---")
print(df.head())

print("\n--- Tipos de Dados e Verificação de Nulos ---")
print(df.info())

# -------------------------------------------------------------
# 3. Tratamento/Preparação dos Dados - Lógica e Algoritmos
# -------------------------------------------------------------

# Coluna TotalCharges: corrigir espaços em branco e nulos.
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

# Lógica de imputação (preenchimento): usando a MEDIANA.
mediana_totalcharges = df['TotalCharges'].median()
df['TotalCharges'] = df['TotalCharges'].fillna(mediana_totalcharges)

# -------------------------------------------------------------

# Justificativa da Etapa 3.a (Markdown):
# **Lógica para Nulos:** A coluna 'TotalCharges' tinha valores inconsistentes (espaços vazios) que foram convertidos para Nulos (NaN) e, em seguida, preenchidos utilizando a **mediana**. A **mediana** é a medida de tendência central preferível neste pré-processamento, pois é menos sensível a *outliers* (valores muito altos ou muito baixos) do que a média, preservando assim a distribuição real dos dados para o algoritmo.00000
# -------------------------------------------------------------
# 1. Codificar a variável alvo (Churn: 'Yes' vira 1, 'No' vira 0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 2. Excluir colunas desnecessárias
df = df.drop(columns=['customerID'])

# 3. One-Hot Encoding: Transforma colunas de texto em colunas binárias (0 ou 1)
df_dummies = pd.get_dummies(df)

# Separação das Features (X) e do Alvo (y)
X = df_dummies.drop(columns=['Churn']) 
y = df_dummies['Churn']

# -------------------------------------------------------------
# Justificativa da Etapa 3.b (Markdown):
# **Algoritmo de Codificação (One-Hot Encoding):** O método `pd.get_dummies` foi aplicado para transformar as variáveis categóricas (texto) em formato binário. Isso é crucial, pois impede que o algoritmo de Machine Learning atribua uma **relação ordinal (de ordem)** que não existe entre as categorias (ex: que um tipo de serviço seja numericamente "melhor" que outro), garantindo que apenas a presença ou ausência da característica seja considerada.
# -------------------------------------------------------------00

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -------------------------------------------------------------
# 4. Análise Exploratória (Balanceamento)
# -------------------------------------------------------------

churn_counts = df['Churn'].value_counts(normalize=True) * 100
print("--- Proporção de Clientes (Churn/Não-Churn) ---")
print(f"Não-Churn (0): {churn_counts[0]:.2f}%")
print(f"Churn (1): {churn_counts[1]:.2f}%")

# -------------------------------------------------------------
# Justificativa da Etapa 4 (Markdown):
# **Impacto do Desbalanceamento:** A análise mostra um desbalanceamento significativo (cerca de 73% Não-Churn contra 27% Churn). Logicamente, um modelo treinado com dados desbalanceados pode ser **viciado** em prever a classe majoritária (Não-Churn), resultando em uma alta Acurácia, mas baixa capacidade de realmente identificar os clientes que vão sair. Isso reforça a necessidade de priorizar o Recall nas métricas de avaliação.
# -------------------------------------------------------------

# -------------------------------------------------------------
# 5. Modelagem do Algoritmo (Mineração)
# -------------------------------------------------------------

# Dividir os dados em conjuntos de TREINO (80%) e TESTE (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o Modelo de Regressão Logística
modelo = LogisticRegression(max_iter=500, solver='liblinear') # Usando liblinear para estabilidade
modelo.fit(X_train, y_train)

# Fazer Previsões
y_predito = modelo.predict(X_test)

# -------------------------------------------------------------
# Justificativa da Etapa 5 (Markdown):
# **Algoritmo Escolhido (Regressão Logística):** Escolhemos a Regressão Logística por ser um classificador robusto para problemas binários. Seu funcionamento algorítmico depende da **função Sigmoide** ($\sigma(z) = \frac{1}{1 + e^{-z}}$), que transforma o resultado linear da equação do modelo em um valor entre 0 e 1, que é interpretado como a **probabilidade** de Churn.
# -------------------------------------------------------------

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# -------------------------------------------------------------
# 6. Análise dos Resultados
# -------------------------------------------------------------

print("--- Relatório Detalhado de Classificação (Acurácia, Precisão, Recall, F1) ---\n")
print(classification_report(y_test, y_predito))

# -------------------------------------------------------------
# Justificativa da Etapa 6.a (Markdown):
# **Métrica Lógica Prioritária (Recall):** No objetivo de retenção de clientes, o **Recall** (Taxa de Verdadeiros Positivos) deve ser priorizado. Ele mede a proporção de clientes que *realmente* iriam sair (Churn) e foram corretamente identificados pelo modelo. Logicamente, o custo de perder um cliente (Falso Negativo) é maior do que o custo de uma ação de retenção desnecessária (Falso Positivo), fazendo do Recall a métrica mais alinhada ao objetivo de negócio.
# -------------------------------------------------------------

# -------------------------------------------------------------
# 6.b) Otimização (Validação Cruzada)
# -------------------------------------------------------------

# Aplicação da Validação Cruzada com 5 Folds, otimizando pela métrica de Recall
scores_cv = cross_val_score(modelo, X, y, cv=5, scoring='recall')

print(f"\n--- Validação Cruzada (Cross-Validation) ---")
print(f"Scores de Recall por Fold (k=5): {scores_cv}")
print(f"Recall Médio: {scores_cv.mean():.4f}")

# -------------------------------------------------------------
# Justificativa da Etapa 6.b (Markdown):
# **Lógica Algorítmica da Validação Cruzada (K-Fold):** Esta técnica divide o dataset em 5 partes (folds). O modelo é treinado 5 vezes, usando em cada rodada K-1 partes para treino e a parte restante para teste. Esta abordagem garante que a avaliação do modelo seja **mais robusta e imparcial**, pois elimina a dependência de uma única divisão aleatória de treino/teste, validando a capacidade de generalização do algoritmo em diferentes subconjuntos de dados.
# -------------------------------------------------------------
# -------------------------------------------------------------
# 7. Deploy (Simulado)
# -------------------------------------------------------------

# Pegamos as colunas em que o modelo foi treinado (o modelo precisa de todas elas)
colunas_treinadas = list(X.columns)

def prever_novo_cliente(dados_cliente: dict, modelo: LogisticRegression, colunas_dummies: list):
    """
    Recebe os dados de um cliente, aplica o pré-processamento (codificação) e retorna a previsão de Churn.
    """
    # 1. Aplicar One-Hot Encoding no novo dado (o novo dado vira uma linha de DataFrame)
    novo_df = pd.DataFrame([dados_cliente])
    
    # 2. Replicar a codificação binária (dummies) nos dados do novo cliente
    # Nota: Este passo assume que o novo dado já tem as colunas codificadas em 0 ou 1
    
    # 3. Alinhar colunas: Garante que o novo dado tem TODAS as 
    #    mesmas colunas que o modelo foi treinado (preenchendo com 0 onde não tem)
    diferenca_colunas = set(colunas_dummies) - set(novo_df.columns)
    for col in diferenca_colunas:
        novo_df[col] = 0

    # 4. Selecionar apenas as colunas que o modelo espera, na ordem correta
    dado_para_prever = novo_df[colunas_dummies].copy()

    # 5. Fazer a previsão de probabilidade e classificação
    # Note: O .predict_proba dá a chance (probabilidade) do Churn (classe 1)
    probabilidade_churn = modelo.predict_proba(dado_para_prever)[:, 1][0]
    
    # 6. Definir a decisão final com base em um limite de 50%
    decisao_final = 'Churn (Alto Risco - Ação de Retenção Recomendada)' if probabilidade_churn >= 0.5 else 'Não-Churn (Baixo Risco)'

    return probabilidade_churn, decisao_final

# -------------------------------------------------------------
# Simulação com um Cliente de Alto Risco (Exemplo)
# -------------------------------------------------------------

# Exemplo de dados de um novo cliente (após o pré-processamento de um sistema externo):
dados_exemplo_alto_risco = {
    # Exemplo: Mulher, não-Senior, Sem Parceiro, Sem Dependentes, 1 mês de contrato, Fibra Ótica, Sem Segurança, Checagem Eletrônica
    'gender_Female': 1, 'SeniorCitizen': 0, 'Partner_No': 1, 'Dependents_No': 1, 
    'tenure': 1, 'PhoneService_Yes': 1, 'MultipleLines_No': 1, 
    'InternetService_Fiber optic': 1, 'OnlineSecurity_No': 1, 'OnlineBackup_No': 1, 
    'DeviceProtection_No': 1, 'TechSupport_No': 1, 'StreamingTV_No': 1, 
    'StreamingMovies_No': 1, 'Contract_Month-to-month': 1, 'PaperlessBilling_Yes': 1, 
    'PaymentMethod_Electronic check': 1, 'MonthlyCharges': 70.0, 'TotalCharges': 70.0
}

prob, decisao = prever_novo_cliente(dados_exemplo_alto_risco, modelo, colunas_treinadas)

print(f"\n--- Simulação de Deploy para Novo Cliente ---")
print(f"Probabilidade de Churn: {prob*100:.2f}%")
print(f"Decisão do Modelo: {decisao}")

# -------------------------------------------------------------
# Justificativa do Deploy (Markdown):
# **Lógica de Pipeline:** A função simula o pipeline em produção. Sua lógica crucial é garantir que os novos dados de entrada sejam submetidos ao **mesmo pré-processamento e codificação** (One-Hot Encoding e alinhamento de colunas) usados no treinamento. Isso é fundamental para que o modelo receba as *features* no formato exato que espera e possa fazer uma previsão válida e consistente.
# -------------------------------------------------------------

--- 10 Primeiras Linhas (Amostra de Dados) ---
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport S

In [14]:
# Célula 2: Continuação do Fluxo

import pandas as pd
import numpy as np

# ⚠️ CORREÇÃO CRUCIAL: RECARREGA O DATAFRAME ORIGINAL
# Isso evita o erro de ter apenas uma classe (0) no modelo.fit.
url_dados = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url_dados)

# -------------------------------------------------------------
# 3. Tratamento/Preparação dos Dados - Lógica e Algoritmos
# -------------------------------------------------------------

# Lógica de imputação (preenchimento): usando a MEDIANA.
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
mediana_totalcharges = df['TotalCharges'].median()
df['TotalCharges'] = df['TotalCharges'].fillna(mediana_totalcharges)

# Justificativa da Etapa 3.a (Markdown):
# **Lógica para Nulos:** A coluna 'TotalCharges' tinha valores inconsistentes que foram preenchidos utilizando a **mediana**. A **mediana** é a medida de tendência central preferível neste pré-processamento, por ser menos sensível a *outliers*.

# 1. Codificar a variável alvo (Churn: 'Yes' vira 1, 'No' vira 0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Garante que Churn não tenha NaNs e é um inteiro
df['Churn'] = df['Churn'].fillna(0).astype(int) 

# 2. Excluir colunas desnecessárias
df = df.drop(columns=['customerID'])

# 3. One-Hot Encoding
df_dummies = pd.get_dummies(df)

# Separação das Features (X) e do Alvo (y)
X = df_dummies.drop(columns=['Churn']) 
y = df_dummies['Churn']

# Justificativa da Etapa 3.b (Markdown):
# **Algoritmo de Codificação (One-Hot Encoding):** O método `pd.get_dummies` foi aplicado para transformar as variáveis categóricas (texto) em formato binário. Isso é crucial, pois impede que o algoritmo atribua uma **relação ordinal** incorreta entre as categorias.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# X e Y já estão definidos na Célula 2.

# -------------------------------------------------------------
# 3.c) Algoritmo de Normalização/Escalonamento
# -------------------------------------------------------------
colunas_numericas = ['tenure', 'MonthlyCharges', 'TotalCharges']

scaler = MinMaxScaler()

# Aplica Normalização nas colunas numéricas (Força float para estabilidade)
X[colunas_numericas] = X[colunas_numericas].astype(float)
X[colunas_numericas] = scaler.fit_transform(X[colunas_numericas])

# Justificativa da Etapa 3.c (Markdown):
# **Necessidade Algorítmica da Normalização:** Aplicamos o **MinMaxScaler** para escalonar as colunas numéricas. A normalização garante que todas as *features* contribuam igualmente para o treinamento, prevenindo vieses.

# -------------------------------------------------------------
# 4. Análise Exploratória (Balanceamento)
# -------------------------------------------------------------
# ⚠️ GARANTINDO QUE DF SEJA O DATAFRAME COM CHURN 0/1 (do final da Célula 2)
# Replicamos o df com as mudanças necessárias apenas para fins de visualização do balanceamento
df['Churn'] = df['Churn'].fillna(0).astype(int) 
churn_counts = df['Churn'].value_counts(normalize=True) * 100

print("--- Proporção de Clientes (Churn/Não-Churn) ---")
print(f"Não-Churn (0): {churn_counts.get(0, 0.0):.2f}%")
print(f"Churn (1): {churn_counts.get(1, 0.0):.2f}%")

# Justificativa da Etapa 4 (Markdown):
# **Impacto do Desbalanceamento:** A análise mostra um desbalanceamento significativo. Isso reforça a necessidade de priorizar o **Recall** nas métricas de avaliação.

# -------------------------------------------------------------
# 5. Modelagem do Algoritmo (Mineração)
# -------------------------------------------------------------

# Dividir os dados em conjuntos de TREINO (80%) e TESTE (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o Modelo de Regressão Logística
modelo = LogisticRegression(max_iter=500, solver='liblinear') 
modelo.fit(X_train, y_train)

# Fazer Previsões
y_predito = modelo.predict(X_test)

# Justificativa da Etapa 5 (Markdown):
# **Algoritmo Escolhido (Regressão Logística):** Escolhemos a Regressão Logística. Seu funcionamento algorítmico depende da **função Sigmoide** que transforma o resultado linear em uma probabilidade de Churn.

--- Proporção de Clientes (Churn/Não-Churn) ---
Não-Churn (0): 73.46%
Churn (1): 26.54%


In [12]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# -------------------------------------------------------------
# 6. Análise dos Resultados
# -------------------------------------------------------------

print("--- Relatório Detalhado de Classificação (Acurácia, Precisão, Recall, F1) ---\n")
print(classification_report(y_test, y_predito))

# Justificativa da Etapa 6.a (Markdown):
# **Métrica Lógica Prioritária (Recall):** O **Recall** deve ser priorizado no objetivo de retenção. O custo de perder um cliente (Falso Negativo) é maior do que o custo de uma ação de retenção desnecessária, alinhando o Recall ao objetivo de negócio.

# -------------------------------------------------------------
# 6.b) Otimização (Validação Cruzada)
# -------------------------------------------------------------

# Aplicação da Validação Cruzada com 5 Folds, otimizando pela métrica de Recall
scores_cv = cross_val_score(modelo, X, y, cv=5, scoring='recall')

print(f"\n--- Validação Cruzada (Cross-Validation) ---")
print(f"Scores de Recall por Fold (k=5): {scores_cv}")
print(f"Recall Médio: {scores_cv.mean():.4f}")

# Justificativa da Etapa 6.b (Markdown):
# **Lógica Algorítmica da Validação Cruzada (K-Fold):** Esta técnica garante que a avaliação do modelo seja **mais robusta e imparcial**, pois elimina a dependência de uma única divisão aleatória de treino/teste.

--- Relatório Detalhado de Classificação (Acurácia, Precisão, Recall, F1) ---

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.69      0.59      0.64       373

    accuracy                           0.82      1409
   macro avg       0.77      0.75      0.76      1409
weighted avg       0.81      0.82      0.82      1409


--- Validação Cruzada (Cross-Validation) ---
Scores de Recall por Fold (k=5): [0.55614973 0.57754011 0.51336898 0.55764075 0.53208556]
Recall Médio: 0.5474


In [13]:
# -------------------------------------------------------------
# 7. Deploy (Simulado)
# -------------------------------------------------------------

# Reutilizando objetos do modelo e do scaler da Célula 3
colunas_treinadas = list(X.columns)
colunas_numericas_brutas = ['tenure', 'MonthlyCharges', 'TotalCharges']

def prever_novo_cliente(dados_cliente: dict, modelo: LogisticRegression, colunas_dummies: list, scaler_model: MinMaxScaler):
    """
    Recebe os dados de um cliente, aplica todas as etapas de pré-processamento (codificação e normalização) 
    e retorna a previsão de Churn.
    """
    novo_df = pd.DataFrame([dados_cliente])
    
    # Aplicar a Normalização/Escalonamento
    dados_para_normalizar = novo_df[colunas_numericas_brutas].astype(float).copy()
    novo_df[colunas_numericas_brutas] = scaler_model.transform(dados_para_normalizar)

    # Alinhar colunas (One-Hot Encoding)
    diferenca_colunas = set(colunas_dummies) - set(novo_df.columns)
    for col in diferenca_colunas:
        novo_df[col] = 0

    dado_para_prever = novo_df[colunas_dummies].copy()

    # Previsão
    probabilidade_churn = modelo.predict_proba(dado_para_prever)[:, 1][0]
    decisao_final = 'Churn (Alto Risco - Ação de Retenção Recomendada)' if probabilidade_churn >= 0.5 else 'Não-Churn (Baixo Risco)'

    return probabilidade_churn, decisao_final

# -------------------------------------------------------------
# Simulação com um Cliente de Alto Risco (Exemplo)
# -------------------------------------------------------------

# Exemplo de dados de um novo cliente (em seus valores BRUTOS)
dados_exemplo_alto_risco = {
    'gender_Female': 1, 'SeniorCitizen': 0, 'Partner_No': 1, 'Dependents_No': 1, 
    'tenure': 1, 'PhoneService_Yes': 1, 'MultipleLines_No': 1, 
    'InternetService_Fiber optic': 1, 'OnlineSecurity_No': 1, 'OnlineBackup_No': 1, 
    'DeviceProtection_No': 1, 'TechSupport_No': 1, 'StreamingTV_No': 1, 
    'StreamingMovies_No': 1, 'Contract_Month-to-month': 1, 'PaperlessBilling_Yes': 1, 
    'PaymentMethod_Electronic check': 1, 'MonthlyCharges': 70.0, 'TotalCharges': 70.0
}

# Passamos o objeto 'scaler' treinado para a função
prob, decisao = prever_novo_cliente(dados_exemplo_alto_risco, modelo, colunas_treinadas, scaler)

print(f"\n--- Simulação de Deploy para Novo Cliente ---")
print(f"Probabilidade de Churn: {prob*100:.2f}%")
print(f"Decisão do Modelo: {decisao}")

# Justificativa do Deploy (Markdown):
# **Lógica de Pipeline:** A função simula o pipeline em produção, garantindo que os novos dados sejam submetidos ao **mesmo pré-processamento** (Codificação One-Hot e a Normalização/Escalonamento) usados no treinamento.


--- Simulação de Deploy para Novo Cliente ---
Probabilidade de Churn: 69.39%
Decisão do Modelo: Churn (Alto Risco - Ação de Retenção Recomendada)
